#  Assignment 1 - Language Model Foundations

<div style="padding:15px 20px 20px 20px;border-left:3px solid green;background-color:#e4fae4;border-radius: 20px;color:#424242;">

Welcome to the **1st assignment** for the **CS-552: Modern NLP course**!

- 😀 Name: **< First Last >**
- ✉️ Email: **< first.last >@epfl.ch**
- 🪪 SCIPER: **XXXXXX**

In the first two parts of this assignment, you need to construct a dataset and use it to train language models (LSTM and Transformer);

In the third part, you will finetune language models (RNN-based and Transformer-based Encoder-Decoder) on a text simplification task.

### **Tasks**
- **[PART 1: Data Preprocessing](#1)**
    - [1.1 Data Cleaning](#11)
    - [1.2 Build Vocabulary](#12)
    - [1.2 Get PyTorch Dataset](#13)
- **[PART 2: Training Language Models](#2)**
    - [2.1 Vanilla LSTM](#21)
    - [2.2 Transformer (DistilGPT2)](#22)
- **[PART 3: Finetuning Language Models](#3)**
    - [3.1 Encoder-Decoder Model](#31)
    - [3.2 Transformer (T5)](#32)


### **Deliverables**
- ✅ This Jupyter notebook
- ✅ `data.py`, `modeling.py` file
- ✅ Checkpoints for two LSTM-variant and DistilGPT2 language models (Part 2)
- ✅ Checkpoints for finetuned encoder-decoder and T5 language models (Part 3)
- ✅ `./tensorboard` directory with logs for all trained/finetuned models

Large files such as model checkpoints and logs should be pushed to the repository with Git LFS. You may also find that training the models on a GPU can speed up the process, we recommend using Colab's free GPU service for this. A tutorial on how to use Git LFS and Colab can be found [here](https://github.com/epfl-nlp/cs-552-modern-nlp/blob/main/Exercises/tutorials.md).

</div>

<div style="padding:15px 20px 20px 20px;border-left:3px solid orange;background-color:#fff5d6;border-radius: 20px;color:#424242;">

## How to implement this assignment

Please read carefully the following points. All the information on how to read, implement and submit your assignment is explained in detail below.

1. For this assignment, you will need to implement and fill in the missing code snippets for both the **Jupyter Notebook** `assignment1.ipynb` and the `data.py` and `modeling.py` python files. In the `data.py` and `modeling.py` files, you will add all the Dataset and Model classes you will implement according to the skeleton present in the files. In the notebook, you will use the definitions from those python files to prepare datasets, train and finetune models and add the report at the end.

2. To implement your coding part, you can import the external libraries we provide in the `requirements.txt` file, however, you should not use any other package not included in these requirements. We recommend using **python=3.10** for this assignment

3. At the end of the notebook, you will need to fill in a **report** template, providing the results of your implementation. We provide you with the template for the report, therefore you need to fill in the missing Markdown cells with the requested information. 

4. Along with the `assignment1.ipynb` and the `data.py`, `modeling.py` files, you need to additionally upload model files under the `models/` dir, regarding the following models:
    - the two LSTM-variant models (PART 2)  
    - the trained-from-scratch DistilGPT2 model (PART 2) 
    - the fine-tuned Encoder-Decoder model (PART 3) 
    - the fine-tuned T5 model (PART 3)
    
You will provide test results on all of the model variants according to the report template.

5. Finally, you will need to log your training pipelines using Tensorboard. Please follow the instructions in the `README.md` of the [tensorboard/](tensorboard/README.md) directory.
</div>

In [ ]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [ ]:
# If you are working in Colab, then consider mounting your assignment folder to your drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# and change the path below to point to the assignment folder
# %cd /content/drive/MyDrive/path-to-your-assignment-folder

### Setup

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0" # limiting to one GPU

In [ ]:
!pip install -r requirements.txt

In [ ]:
from datasets import load_dataset
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import evaluate
import gensim
import transformers
import nltk

<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

**TODO🔻: Enter your SCIPER number below!**
     
</div>

In [ ]:
import re
import random
import numpy as np
import torch

## Enter your SCIPER here: ##
SCIPER = ""

try:
    assert re.match("\d{6}", SCIPER)[0] == SCIPER, "Invalid SCIPER given. please enter your correct 6-digit SCIPER number above!"
except:
    print("Invalid SCIPER given. please enter your correct 6-digit SCIPER number above!")

student_seed = int(SCIPER)


"""Set seed for reproducibility."""
random.seed(student_seed)
np.random.seed(student_seed)
torch.manual_seed(student_seed)
torch.cuda.manual_seed_all(student_seed)

---

<a name="1"></a>
# PART 1: Data Preprocessing

We will train a language model using the `wikitext-103` dataset.

> `wikitext-103` dataset is a large collection of articles with verified
good and featured quality from Wikipedia. We will use the open-source dataset from Huggingface (https://huggingface.co/datasets/wikitext).

In [ ]:
from utils import *
from test_A1 import *
from data import filter_by_length, data_clean, count_tokens, build_vocabulary

### Load `wikitext-103` Dataset

In [ ]:
wikitext_dataset = load_dataset("wikitext", 'wikitext-103-v1', split="train")

print(f"Size of the dataset is {len(wikitext_dataset)}")

# You can print some samples to get a sense of the data
print(wikitext_dataset[0])
print(wikitext_dataset[10])
print(wikitext_dataset[100])

<a id="11"></a>
## 🎯 Q1.1: **Data Cleaning**.

We find some problems in the dataset:
- some samples are empty (with no tokens) or too short;
- some contains noisy texts (e.g. `==== <> ====`);
- some have non-English texts.

So we need a data cleaning before feeding it to the model.

***Test:*** After each function, you can test your implementation by running a simple test case. (Note: passing these simple cases do not necessarily mean your implementation is 100% correct.)

<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

**We will perform 4-step data cleaning:**

- **Step 1**: filter out sequences shorter than `min_len=100` or longer than `max_len=128`;
- **Step 2**: filter out sequences with particular pattern `= * =\n`, where `*` denotes any possible sequences;
- **Step 3**: filter out Non-English sequences;
- **Step 4**: lowercase all sequences.

**TODO🔻: Implement `filter_by_length` and `data_clean` function in `data.py`**.

More instructions are provided in the function description.
</div>

In [ ]:
# ETS: take ~1-3mins to run #
from data import data_clean

wikitext_dataset = data_clean(wikitext_dataset, min_len=100, max_len=128)

In [ ]:
test_data_clean(wikitext_dataset)

<a id="12"></a>
## 🎯 Q1.2: **Build Vocabulary**.


<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

**We have to build a vocabulary dictionary for the cleaned `wikitext_dataset` following four steps:**

- **Step 1**: Compute frequency of occurences for each unique token in the dataset;

- **Step 2**: Find a set of rare tokens with frequency lower than `min_freq=5`. Replace them with `unk_token='<unk>'`;

- **Step 3**: Filter out sequences with more than 15\% rare tokens. (*We implemented this for you.*)

- **Step 4**: Recompute the token frequency to get final vocabulary dict.

**TODO🔻: Implement `count_tokens` and `build_vocabulary` function in `data.py`**.

More instructions are provided in the function description.
</div>

In [ ]:
# ETS: take ~1-3mins to run #
from data import build_vocabulary

wikitext_dataset, token_freq_dict = build_vocabulary(wikitext_dataset, min_freq=5, unk_token='<unk>')

In [ ]:
test_vocab_build(wikitext_dataset, token_freq_dict)

**Let's look at the vocabulary distribution with a histogram:**

- we can observe that tokens with very low frequency are filtered out :)

In [ ]:
plt.hist(token_freq_dict.values(), bins=10**np.linspace(0,6,33))
plt.xscale("log")

<a id="13"></a>
## 🎯 Q1.3: **Build Pytorch Dataset**.

<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

**We build a Pytorch Dataset using cleaned `wikitext_dataset`:**

- **Step 1**: we have to add some special tokens for each sequence:
  - `<start>`: add to the start of a sequence;
  - `<stop>`: add to the end of a sequence;
  - `<pad>`: padding token. add padding token until the length of the sequence reach `MAX_SEQ_LENGTH`.

- **Step 2**: when we fetch a sequence in the dataset by indexing, it should return a sequence of token indices.

**TODO🔻: Implement `__init__` and `__getitem__` function in `RNNDataset` class in `data.py`**.

Detailed instructions are provided in the function description.
</div>


In [ ]:
from data import RNNDataset

# ETS: take ~1min to run #
MAX_SEQ_LENGTH = 128
rnn_dataset = RNNDataset(dataset=wikitext_dataset,
                         max_seq_length=MAX_SEQ_LENGTH)

In [ ]:
test_rnn_dataset(rnn_dataset)

<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

**Once we create the `rnn_dataset`, we have to split it into train/test dataset, and build a DataLoader for each::**

- **Step 1**: split `rnn_dataset` into train/test datasets, with `test_ratio=0.1`:

- **Step 2**: Build pytorch dataloader for train/test dataset with `batch_size=8`.

**TODO🔻: Implement `get_dataloader` function in `data.py`**.

Detailed instructions are provided in the function description.
</div>


In [ ]:
from data import get_dataloader

# ETS: ~1min #
train_dataloader, test_dataloader = get_dataloader(rnn_dataset, test_ratio=0.1)

In [ ]:
test_dataloaders(train_dataloader, test_dataloader)

<div style="padding:15px 15px 15px 15px;border-left:3px solid #8e7cc3;background-color:#e4e1eb;border-radius: 15px;color:#424242;">

🎉 **Excellent work!** By this point, we have all core building blocks for Part 2 :)
   
- [X] `rnn_dataset`: A `Dataset` obj with the data, the vocabulary, the pad index, and mapping from tokens to indices;
- [X] `train_dataloader`: A `DataLoader` obj with training sequences;
- [X] `test_dataloader`: A `DataLoader` obj with testing sequences.

_Tip: Try to familiarize with functionalities and attributes they provide._
    
</div>

---

<a id="2"></a>
# PART 2:  Training Language Models

#### Language Model: a probabilistic model of a sequence of tokens.

🔵 **What?**

Language modeling (LM) is the use of various statistical and probabilistic techniques to determine the probability of a given sequence of words occurring in a sentence. Language models analyze bodies of text data to provide a basis for their word predictions. They are used in natural language processing (NLP) applications, particularly ones that generate text as an output. Some of these applications include, machine translation and question-answering.

🟡 **How?**

There are several different probabilistic approaches to modeling language, which vary depending on the purpose of the language model. From a technical perspective, the various types differ by the amount of text data they analyze and the math they use to analyze it (architecture). Some LMs we've already seen and will learn about during lectures are n-gram / count-based models, Recurrent Neural Networks (RNNs), and Transformer models.

🟣 **Why?**

Language modeling is crucial in modern NLP applications. It is the reason that machines can understand qualitative information. Each language model type, in one way or another, turns qualitative information into quantitative information. This allows people to communicate with machines as they do with each other to a limited extent. It is used directly in a variety of industries including tech, finance, healthcare, transportation, legal, military and government. Additionally, it's likely most people reading this have interacted with a language model in some way at some point in the day, whether it be through Google search, an autocomplete text function or engaging with a voice assistant.

ℹ️ Source: [Original article](https://www.techtarget.com/searchenterpriseai/definition/language-modeling#:~:text=Language%20models%20determine%20word%20probability,predict%20or%20produce%20new%20sentences.)



<div style="padding:15px 15px 15px 15px;border-left:3px solid gray;background-color:#F3F3F3;border-radius: 15px;color:#424242;">

**In this part, you will train your own language models using the dataset created in Part 1.**

You need to implement **3 different model variants**;
train and evaluate their perplexity.
    
| Model | Variant | Description |
|:---- |:----- | :----- |
|**LSTM** | Token embeddings trained from scratch | An LSTM model with a trainable token Embedding layer <br>that will be initialized randomly and trained from scratch along with the LM. |
| | Pre-trained token embeddings| An LSTM model with pre-trained GloVe embeddings as input <br>that will be ***frozen*** while the LM is training. |
| **Transformer** | Trained from scratch | A Transformer based model that follows the architecture of [DistilGPT2](https://huggingface.co/distilgpt2). |
    
</div>

<a id="21"></a>
## 🎯 Q2.1: **Vanilla LSTM**.


<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

**First, we train a vanilla LSTM on our train/test dataLoaders:**

- **Step 1**: Build the `VanillaLSTM` class, which takes:
  - `vocab_size`: size of vocabulary,
  - `embedding_dim`: dimension of token embedding,
  - `hidden_dim`: hidden dimension of lstm layer;
  - `num_layers`: number of stacked lstm layers;
  - `embedding_weights`: pretrained embedding weights. Train from scratch if set to `None`. Default: None;
  - `freeze_embeddings`: whether to freeze pretrained embeddings. Only be functional when `embedding_weights` is not None.

- **Step 2**: Train the LSTM model with `train_dataloader`. We usually run only one epoch for language modelling task to prevent overfitting;

- **Step 3**: Evaluate the LSTM by perplexity scores on `test_dataloader`.

**TODO🔻: Implement `VanillaLSTM`, `train_lstm` and `test_lstm` function in `modeling.py`**.

Detailed instructions are provided in the function description.
</div>


#### **LSTM from Scratch**

We here train the LSTM embedding weights from scratch. The dimension of embedding is defined as `embedding_dim`.

We set the `embedding_dim` as 100 here to compare with gensim pretrained embeddings.

In [ ]:
from modeling import VanillaLSTM

# Define the parameters to VanillaLSTM
## Hint: what do we have from Part1?
vocab_size = len(rnn_dataset.token2idx)
embedding_dim = 100
hidden_dim = 100
num_layers = 2
dropout_rate = 0.15
lr = 1e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = VanillaLSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate).to(device)

In [ ]:
test_lstm_scratch(model)

##### Train&Evaluation (LSTM-scratch)

In [ ]:
# Define optimizer and criterion (loss function)
## Hint: what do we have from Part1?
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(ignore_index=rnn_dataset.pad_idx)

In [ ]:
from modeling import train_lstm, test_lstm

# ETS: take ~30mins for training #
train_lstm(model, train_dataloader, optimizer, criterion, device=device, tensorboard_path="./tensorboard/lstm_scratch")
torch.save(model.state_dict(), 'models/lstm_scratch.pt')
test_lstm(model, test_dataloader, criterion, device=device)

#### **LSTM with Pretrained Embeddings**

We first download the gensim embeddings, then fix the embedding layer in the LSTM model.

In [ ]:
import gensim.downloader
# Download the "glove-wiki-gigaword-100" embeddings
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100')
initial_embedding_weight = torch.nn.Embedding(len(rnn_dataset.token2idx),
                                              embedding_dim).weight.detach().numpy()

# Get the pretrained embeddings from gensim for each tokens
for word_type, idx in rnn_dataset.token2idx.items():
  if word_type in glove_vectors.key_to_index.keys():
      initial_embedding_weight[idx] = glove_vectors[word_type]

In [ ]:
# Define the parameters to VanillaLSTM
## Hint: Use the same params as LSTM_scratch
vocab_size = len(rnn_dataset.token2idx)
embedding_dim = 100
hidden_dim = 100
num_layers = 2
dropout_rate = 0.15
lr = 1e-3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = VanillaLSTM(vocab_size, embedding_dim, hidden_dim,
                    num_layers, dropout_rate=dropout_rate,
                    embedding_weights=initial_embedding_weight,
                    freeze_embeddings=True).to(device)

## Define optimizer and criterion
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(ignore_index=rnn_dataset.pad_idx)

In [ ]:
test_lstm_pretrained(model)

##### Train&Evaluation (LSTM-pretrained)

In [ ]:
# ETS: take ~30mins for training #
train_lstm(model, train_dataloader, optimizer, criterion, device=device, tensorboard_path="./tensorboard/lstm_pretrained")
torch.save(model.state_dict(), 'models/lstm_pretrained.pt')
test_lstm(model, test_dataloader, criterion, device=device)

#### **Compare LSTM-scratch and LSTM-pretrained?**

**TODO🔻: compare the *performance* and *training efficiency* between `LSTM-scratch` and `LSTM-pretrained`**.



ans: [Enter your findings here. ]

<a id="22"></a>
## 🎯 Q2.2: **Train a Transformer: DistillGPT2**.


<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

**We train a DistillGPT2 on our train/test dataLoaders:**

- **Step 1**: Rebuild the datasets and dataloaders using huggingface tokenizer;

- **Step 2**: Train a DistilGPT2 efficiently using Trainer form `transformers`.

- **Step 3**: Evaluate perplexity scores on `test_dataloader`.

**TODO🔻: follow the instructions below**.
</div>

In [ ]:
from transformers import AutoConfig, AutoModelWithLMHead, AutoTokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments

model_name = "distilgpt2"
tokenizer_checkpoint = "distilgpt2"

In [ ]:
MAX_SEQ_LENGTH = 128

# Load model from scratch
model_config = AutoConfig.from_pretrained(model_name)
gpt2_scratch_model = AutoModelWithLMHead.from_config(model_config)
gpt_tokenizer = AutoTokenizer.from_pretrained(model_name)
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

#### **Tokenize Datasets**
 Huggingface provides pretrained tokenizer for `DistilGPT2`, so we have to tokenize our dataset with this tokenizer.

**TODO🔻: Implement `get_encoded_dataset`**.

In [ ]:
def get_encoded_dataset(wikitext_dataset):
  # TODO: Tokenize wikitext_dataset with DistilGPT2 tokenizer. Padding to `max_length`.
  encoded_dataset = ...
  # TODO: Filter out samples longer than MAX_SEQ_LENGTH=128
  encoded_dataset = ...

  encoded_dataset = encoded_dataset.map(
    lambda example: {"input_ids": example['input_ids'][:MAX_SEQ_LENGTH],
                     "attention_mask": example["attention_mask"][:MAX_SEQ_LENGTH]})
  encoded_dataset = encoded_dataset.remove_columns("text")
  encoded_dataset = encoded_dataset.with_format("torch")
  encoded_dataset = encoded_dataset.map(lambda example:{"labels": example["input_ids"]})
  return encoded_dataset

## ETS: ~10mins to run.
encoded_dataset = get_encoded_dataset(wikitext_dataset)
transformer_train_dataset, transformer_test_dataset = torch.utils.data.random_split(encoded_dataset,
                                                               [int(len(encoded_dataset)*0.9), len(encoded_dataset)-int(len(encoded_dataset)*0.9)])

#### **Build Trainer**
 Huggingface provides very efficient `Trainer` wrapper for language model training and evaluation.

**TODO🔻: Run the following blocks to define the training arguements and trainer, then start training simply by `trainer.train()`**.

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=gpt_tokenizer, mlm=False)
training_args = TrainingArguments(
    output_dir=f"{model_name}-wikitext103",
    evaluation_strategy = "steps",
    num_train_epochs = 1.0,
    logging_steps=500,
    learning_rate=2e-5,
    save_steps=1000,
    weight_decay=0.01,
    report_to="tensorboard",
    logging_dir="./tensorboard/distilgpt2-scratch")

trainer = Trainer(
    model=gpt2_scratch_model,
    args=training_args,
    tokenizer=gpt_tokenizer,
    train_dataset=transformer_train_dataset,
    eval_dataset=transformer_test_dataset,
    data_collator=data_collator,)

In [ ]:
# ETS: ~20mins to run.
trainer.train()
trainer.save_model('models/distilgpt2_scratch')

**TODO🔻: Run evaluation by `trainer.evaluate()`**.

In [ ]:
# ETS: ~1min to run.
eval_result = trainer.evaluate()
perplexity_from_scratch = math.exp(eval_result["eval_loss"])
print(f"The perplexity on the test dataset is {perplexity_from_scratch:.3f}")

#### **Compare to the Pretrained DistilGPT2**

**TODO🔻: Load the pretrained `DistilGPT2` checkpoint from Huggingface.
We only run evaluation of the `trainsformer_test_dataset` here. Compare the performance between `gpt2_scratch_model` and `gpt2_pretrained_model`.**.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Trainer, TrainingArguments

model_id = "distilgpt2"
gpt2_pretrained_model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer_pretrained_gpt = AutoTokenizer.from_pretrained(model_id)
tokenizer_pretrained_gpt.pad_token = tokenizer_pretrained_gpt.eos_token


In [ ]:
training_args = TrainingArguments(
    output_dir=f"pretrained_{model_id}-wikitext103",
    evaluation_strategy = "steps",
    num_train_epochs = 1.0,
    logging_steps=500,
    learning_rate=2e-5,
    save_steps=1000,
    weight_decay=0.01,
    report_to="tensorboard",
    logging_dir="./tensorboard/distilgpt2-pretrained")

pretrained_trainer = Trainer(
    model=gpt2_pretrained_model,
    args=training_args,
    tokenizer=tokenizer_pretrained_gpt,
    train_dataset=transformer_train_dataset,
    eval_dataset=transformer_test_dataset,
    data_collator=data_collator,)

In [ ]:
# ETS: ~1min.
eval_result = pretrained_trainer.evaluate()
perplexity_pretrained_model = math.exp(eval_result["eval_loss"])
print(f"The perplexity of pretrained {model_id} on the test dataset is {perplexity_pretrained_model:.3f}")

<div style="padding:15px 15px 15px 15px;border-left:3px solid #8e7cc3;background-color:#e4e1eb;border-radius: 15px;color:#424242;">
    
🎉  **Excellent work!** We have completed Langauge Modelling tasks!

#### Part 2 - Checklist
Here are the core building blocks you created and that you will need for Part 3:
   
- [X] LSTM-variants checkpoints.
- [X] LSTM-variants ppl scores.
- [X] Transformer-variants ppl scores.

</div>

Just before you move on to the Part 3, let's save a few things which we will use in the Part 3 so that you don't have to run Part 1 and 2 every time.

#### Save wikitext vocabulary

In [ ]:
from utils import save_binary

save_binary(rnn_dataset.dataset_vocab, "wikitext_vocab.pkl")

---

<a id="3"></a>
# PART 3: Fine-tune on the Text Simplification task 🚀

In this part we will fine-tune and test language models on the downstream task of text simplification (a.k.a text compression). In this task, given a piece of text, the goal is to simplify (or compress) this text by extracting the most important information from it. We will consider encoder-decoder models built on top of the LSTM models you built in Part 2 and also pretrained transformer models from Huggingface.
 
Hence, there will be two sections in this final part of the assignment:
- [3.1 Finetune a custom Encoder-Decoder model on text simplification task](#32)
- [3.2 Finetune pretrained T5 model (an encoder-decoder) model on text simplification task](#33)

### Dataset
In this part, we will use a sentence compression [dataset](https://huggingface.co/datasets/embedding-data/sentence-compression) available on Huggingface. Each row in this dataset contains two sentences: the original sentence and the simplified one.

An example from this dataset is the following:

**Original sentence:** "The JSE kept toying with an all time high by midday today as resources continued to fuel the bourse."

**Simplified sentence:** "JSE keeps toying with all time high"

This dataset contains around 180k examples, but we will only use a subset of it.

### Load and preprocess sentence-compression dataset

In [ ]:
# We will use NLTK to tokenize the text
from datasets import load_dataset
import nltk
nltk.download('punkt')

In [ ]:
scomp = load_dataset("embedding-data/sentence-compression")

Let's inspect the dataset. Note that this dataset only contains a train split (we will create our own train, validation and test sets from this split later)

In [ ]:
scomp["train"][0]

First to make our dataset and sentences smaller, let's filter examples where the original sentence has more than 10 and less than 100 words.

<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

🎯 Goal: Keep only the examples that has more than 10 and less than 100 words in its original sentence.

</div>

In [ ]:
scomp_small = ...

Next, let's define our train, validation and test sets of size 10k, 1k, and 1k respectively and also get rid of the 'set' attribute.

In [ ]:
scomp_train, scomp_val, scomp_test = scomp_small["train"][:10000]['set'], scomp_small["train"][10000:11000]['set'], scomp_small["train"][11000:12000]['set']

<a id="22"></a>
## 🎯 Q3.1: **Finetune custom encoder-decoder model on sentence simplification task**.


<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px; color:#424242;">

**In this section, we will build a model based on the [encoder-decoder architecture](https://arxiv.org/abs/1409.3215) and finetune it on our text simplification task. As you probably know by now that vanilla encoder-decoder models do not perform well when we have long-range dependencies. Therefore, we will build one where the decoder uses [additive attention](https://arxiv.org/abs/1409.0473) over encoder outputs. You will need to finish the following main steps:**

- **Step 1**: Implement a Tokenizer

- **Step 2**: Build a Pytorch Dataset

- **Step 3**: Implement an Encoder-Decoder Model

- **Step 4**: Finetune the model on the task

- **Step 5**: Test and evaluate model performance

**TODO🔻: follow the instructions below**.
</div>

### Implement a tokenizer
We will first implement a tokenizer for our dataset. This tokenizer will take our vocabulary from the Part 1 and will have methods to encode a given text and decode a given text ids.

<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

🎯 Goal:  **Go to `data.py` and implement the `CustomTokenizer` class.**
    
</div>

Let's first load our wikitext vocabulary from Part 1.

In [ ]:
from utils import load_binary

wikitext_vocab = load_binary("wikitext_vocab.pkl")

Initialize the custom tokenizer with this vocabulary.

In [ ]:
from data import CustomTokenizer

custom_tokenizer = CustomTokenizer(vocab=wikitext_vocab)

### Implement a pytorch dataset
Now we will implement a Pytorch dataset that will represent our data in a suitable format for our model fine-tuning later.

<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

🎯 Goal:  **Go to `data.py` and implement the `SCompDataset` class.**
    
</div>

Initialize train, validation and test pytorch datasets.

In [ ]:
from data import SCompDataset

MAX_SEQ_LENGTH = 128
scomp_train_ds = SCompDataset(scomp_train, custom_tokenizer, MAX_SEQ_LENGTH)
scomp_val_ds = SCompDataset(scomp_val, custom_tokenizer, MAX_SEQ_LENGTH)
scomp_test_ds = SCompDataset(scomp_test, custom_tokenizer, MAX_SEQ_LENGTH)

Initialize dataloaders for our datasets respectively.

In [ ]:
from torch.utils.data import DataLoader

# feel free to change batch size according to your GPU memory
scomp_train_dataloader = DataLoader(scomp_train_ds, batch_size=32, shuffle=True)
scomp_val_dataloader = DataLoader(scomp_val_ds, batch_size=32, shuffle=True)
scomp_test_dataloader = DataLoader(scomp_test_ds, batch_size=32, shuffle=True)

### Implement an Encoder-Decoder model
In this section, you will implement a model based on Encoder-Decoder architecture where the encoder will be one of the LSTM variants you implemented in Part 2. You will specifically implement an attention-based encoder-decoder where decoder attends over the encoder outputs to allow it to handle long range dependencies well.

<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

🎯 Goal:  **Go to `modeling.py` and implement the `Encoder`, `AdditiveAttention`, `Decoder` and `EncoderDecoder` classes.**
    
</div>

Next we will load a pretrained encoder from Part 2. Choose the encoder that had best performance based on the perplexity metric.

In [ ]:
from modeling import VanillaLSTM

vocab_size = len(wikitext_vocab)
embedding_dim = 100
hidden_dim = 100
num_layers = 2
dropout_rate = 0.15

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

pretrained_encoder = VanillaLSTM(vocab_size, embedding_dim, hidden_dim,
                                  num_layers, dropout_rate=dropout_rate).to(device)

# TODO: Load the pretrained model from the file
pretrained_encoder.load_state_dict(...)

Next we will need to implement training, evaluation and generation methods.

<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

🎯 Goal:  **Go to `modeling.py` and implement the `seq2seq_train`, `seq2seq_eval` and `seq2seq_generate` functions.**
    
</div>

Now we are ready to finetune our encoder-decoder models on the sentence simplification dataset.

In [ ]:
from modeling import EncoderDecoder

lr = ...
dropout_rate = ...
bos_token_id = custom_tokenizer.bos_token_id
encoder_decoder = EncoderDecoder(hidden_dim, vocab_size, vocab_size, bos_token_id=bos_token_id, dropout_rate=dropout_rate, pretrained_encoder=pretrained_encoder).to(device)
optimizer = optim.Adam(encoder_decoder.parameters(), lr=lr)
criterion = nn.NLLLoss(ignore_index=custom_tokenizer.pad_token_id)
num_params = sum(p.numel() for p in encoder_decoder.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

In [ ]:
from modeling import seq2seq_train

# ETS: ~30 mins to run with a batch size of 32 for 20 epochs
seq2seq_train(model=encoder_decoder,
              train_loader=scomp_train_dataloader,
              eval_loader=scomp_val_dataloader,
              optimizer=optimizer,
              criterion=criterion,
              device=device,
              tensorboard_path="./tensorboard/encoder_decoder")
# save the model
torch.save(encoder_decoder.state_dict(), "models/encoder_decoder.pt")

Now we have the finetuned encoder-decoder model, we will use it to generate simplified text from the test set and compute a rouge score.

In [ ]:
from modeling import seq2seq_generate
test_generations = seq2seq_generate(encoder_decoder, scomp_test_dataloader, custom_tokenizer, device=device)

Inspect the generations. Do they look like reasonable outputs?

In [ ]:
test_generations[:5]

Let's implement an evaluation function for ROUGE score.

**ROUGE score** stands for Recall-Oriented Understudy for Gisting Evaluation. In its simplest form ROUGE score is the quotient of the matching words under the total count of words in reference sentence. Regarding the denominator ROUGE is a recall oriented metric. 

![rouge.png](docs/rouge.png)

**ROUGE-L score** is based on the length of the longest common subsequence (LCS). To counter the disadvantages of a pure recall metric as in ROUGE-N, Rouge-L calculates the weighted harmonic mean (or f-measure) combining the precision score and the recall score.

![rouge_l.png](docs/rouge_l.png)

ℹ️ Source: [Original article](https://clementbm.github.io/theory/2021/12/23/rouge-bleu-scores.html#bleu)

<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

🎯 Goal:  **Go to `modeling.py` and implement the `evaluate_rouge` method.**
    
</div>

Calculate Rouge scores.

In [ ]:
from modeling import evaluate_rouge

print(evaluate_rouge(test_generations))

<a id="22"></a>
## 🎯 Q3.2: **Finetune pretrained T5 model on text simplification task**.


<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

**In this section, we will take a pretrained [T5](https://arxiv.org/abs/1910.10683) model which is also an encoder-decoder model but based on the transformer architecture and finetune it on our task. You will need to do the following main steps:**

- **Step 1**: Load T5 Model from Huggingface

- **Step 2**: Implement Pytorch dataset for this model

- **Step 3**: Run model finetuning

- **Step 4**: Test and evaluate model performance


**TODO🔻: follow the instructions below**.
</div>

Initialize T5 model from Huggingface pretrained model.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "t5-small"
t5_tokenizer = ...
# Make sure to put your model on the GPU
t5_model = ...

Let's define a pytorch dataset for our T5 model.

<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

🎯 Goal:  **Go to `modeling.py` and implement the `SCompT5Dataset` class.**
    
</div>

In [ ]:
from data import ScompT5Dataset
scomp_t5_train_dataset = ScompT5Dataset(scomp_train, t5_tokenizer)
scomp_t5_val_dataset = ScompT5Dataset(scomp_val, t5_tokenizer)
scomp_t5_test_dataset = ScompT5Dataset(scomp_test, t5_tokenizer)

Prepare finetuning arguments

In [ ]:
from transformers import TrainingArguments, Trainer

# create the finetuning trainer
# TODO: set the hyperparameters to reasonable values
training_args = TrainingArguments(
    output_dir=f"finetune_{model_name}-SCOMP",
    evaluation_strategy="epoch",
    per_device_train_batch_size=...,
    per_device_eval_batch_size=...,
    logging_steps=100,
    learning_rate=...,
    num_train_epochs=...,
    save_strategy="no",
    weight_decay=...,
    report_to="tensorboard",
    logging_dir="./tensorboard/t5-scomp")

#### Run the model finetuning

In [ ]:
trainer = Trainer(
    model=t5_model,
    args=training_args,
    train_dataset=scomp_t5_train_dataset,
    eval_dataset=scomp_t5_val_dataset
)

trainer.train()
trainer.save_model(f"models/finetune_{model_name}_scomp")

Now we will load our final finetuned model from the last checkpoint and evaluate it on the test set.

In [ ]:
from transformers import T5ForConditionalGeneration
t5_model = T5ForConditionalGeneration.from_pretrained(f"models/finetune_{model_name}_scomp").to(device)

<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;color:#424242;">

🎯 Goal:  **Go to `modeling.py` and implement the `t5_generate` method.**
    
</div>

In [ ]:
from modeling import t5_generate

test_t5_generations = t5_generate(scomp_t5_test_dataset, t5_model, t5_tokenizer)

Let's inspect the generations. Are they better than the ones generated by the encoder-decoder model? Provide some reasons why one model's generations are better than the other.

In [ ]:
test_t5_generations[:5]

**Answer:**

Finally, let's compute the ROUGE scores.

In [ ]:
from modeling import evaluate_rouge

print(evaluate_rouge(test_t5_generations))

<div style="padding:15px 15px 15px 15px;border-left:3px solid #8e7cc3;background-color:#e4e1eb;border-radius: 15px;color:#424242;">

🎉 Excellent work! You just finished the code implementation parts of the assignment. 

#### Part 3 - Checklist
Here are the elements you will need for the report in Part 4:

- [X] Finetuned encoder-decoder model and its rouge scores.
- [X] Finetuned T5 model and its rouge scores.
</div>

---

<a id="4"></a>
# PART 4: Write your report 📘

Fill in the tables with the respective scores. 

<div style="padding:15px 15px 15px 15px;border-left:3px solid #03befc;background-color:#eff7fe;border-radius: 15px;text-align:center;color:#424242;">

#### Perplexity results on Language Models

| Model - Variant | PPL |
|:--------- | :-----: |
| LSTM Variant A - Embeddings trained from scratch | YOUR SCORE HERE |
| LSTM Variant B - Pre-trained embeddings & frozen | YOUR SCORE HERE |
||||
| DistilGPT2 - Trained from scratch | YOUR SCORE HERE |
| Pre-trained DistilGPT2 | YOUR SCORE HERE |
    
#### Performance scores on Text Simplification
| Model - Variant | ROUGE-1 | ROUGE-2 | ROUGE-L | ROUGE-Lsum |
|:--------- | :-----: | :-----: |  :-----: |  :-----: | 
| Finetuned Encoder-Decoder| YOUR SCORE HERE |YOUR SCORE HERE |  YOUR SCORE HERE |YOUR SCORE HERE |
| Finetuned T5 | YOUR SCORE HERE |YOUR SCORE HERE |  YOUR SCORE HERE |YOUR SCORE HERE |

</div>